In [388]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [389]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_svmlight_file
from sklearn.model_selection import train_test_split
import sys; sys.path.insert(0,'..')
from main import *

In [2]:
X, y = load_svmlight_file('../data/adult/a5a.txt', n_features=123)
X = X.toarray()

In [3]:
svm_arr, tree_arr = data_size_experiment(X, y, rank=True, plots_or_table='table', verbose=True)

Progress:
LINEAR SVM SUBMODELS
Using 20 classifiers and sample proportion of 4
t	Train		Test
1	0.731		0.724
2	0.469		0.454
3	0.684		0.694
4	0.773		0.762
5	0.707		0.688
6	0.589		0.598
7	0.816		0.830
8	0.569		0.566
9	0.714		0.702
10	0.809		0.816
11	0.636		0.634
12	0.746		0.728
13	0.817		0.816
14	0.769		0.752
15	0.691		0.684
16	0.819		0.822
17	0.749		0.730
18	0.717		0.706
19	0.814		0.800
20	0.770		0.746
Done!
DECISION TREE SUBMODELS
Using 20 classifiers and sample proportion of 4
t	Train		Test
1	0.686		0.672
2	0.314		0.322
3	0.686		0.678
4	0.314		0.324
5	0.687		0.676
6	0.314		0.324
7	0.686		0.676
8	0.314		0.328
9	0.686		0.676
10	0.314		0.326
11	0.770		0.756
12	0.770		0.756
13	0.770		0.756
14	0.770		0.756
15	0.770		0.756
16	0.770		0.756
17	0.770		0.756
18	0.770		0.756
19	0.770		0.756
20	0.770		0.756
Done!
1
LINEAR SVM SUBMODELS
Using 20 classifiers and sample proportion of 4
t	Train		Test
1	0.737		0.708
2	0.413		0.390
3	0.713		0.726
4	0.783		0.738
5	0.743		0.742
6	0.626		0.616
7	0.791		0.7

In [4]:
table_results(svm_arr, tree_arr)

Performance on test set:

	acc	std
SVM: 	0.62	0.07
Tree: 	0.72	0.06


## Old W

In [625]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, stratify=y)

X_train = X_train[:700]
y_train = y_train[:700]

In [626]:
W_true = generate_W(y_train)

In [12]:
W = generate_rank_W(X_train, y_train, m=200)

In [14]:
np.mean(np.diag(W)), np.std(np.diag(W))

(0.4694485, 0.077787615)

In [32]:
upp = np.triu(W)
low = np.tril(W)

In [40]:
np.mean(np.abs(upp - low.T)), np.std(np.abs(upp - low.T))

(0.10363916, 0.14904092)

In [45]:
np.mean(np.abs(W - W_true)), np.std(np.abs(W - W_true))

(0.20754885181710883, 0.16431268287343714)

In [57]:
q = np.ones((3, 3))
q[np.diag_indices(3)] = 0
q

array([[0., 1., 1.],
       [1., 0., 1.],
       [1., 1., 0.]])

## New W

In [631]:
# def generate_rank_data(X_train, y_train, m):
#     """ Generates data to train a ranking model with m true labels """

#     """ Set m instead of m_rel """
#     # m_rel = int(m * (m - 1) / 2)

#     # Get indices of upper triangle of nxn matrix
#     n = y_train.size
#     idx = np.triu_indices(n, k=1)
#     idx = np.hstack((idx[0].reshape(-1, 1), idx[1].reshape(-1, 1)))

#     # Randomly select m index pairs
#     save_row = np.random.choice(idx.shape[0], size=m, replace=False)
#     save_idx = idx[save_row]

#     # Fill in new randomly sampled features and label differences
#     X_new = np.full(shape=(2 * m, 3 * X_train.shape[1]), fill_value=np.NaN)
#     y_new = np.full(shape=2 * m, fill_value=np.NaN)
#     k = 0

#     # Fill in two-sided ([i, j] and [j, i]) and delta features
#     """ Two sided and delta features may not help much """
#     for (i,j) in save_idx:

#         X_new[k]     = np.hstack((X_train[i], X_train[j], X_train[i] - X_train[j]))
#         X_new[k + 1] = np.hstack((X_train[j], X_train[i], X_train[j] - X_train[i]))

#         if y_train[i] == y_train[j]:
#             y_new[k]     = 0.5
#             y_new[k + 1] = 0.5

#         elif y_train[i] > y_train[j]:
#             y_new[k]     = 1.0
#             y_new[k + 1] = 0.0

#         elif y_train[i] < y_train[j]:
#             y_new[k]     = 0.0
#             y_new[k + 1] = 1.0

#         k += 2

#     return X_new, y_new, save_idx

In [632]:
# def generate_rank_W(X_train, y_train, m, rounded):
#     """ Generates pairwise comparison matrix W using a learned ranker trained on m true comparisons """

#     # Get m randomly sampled pairs in W to train with
#     X_new, y_new, save_idx = generate_rank_data(X_train, y_train, m)

#     # Fit regressor on given pairwise comparisons
#     """ Change max_depth? """
#     reg = XGBRegressor(max_depth=5, n_estimators=100, objective='reg:squarederror')
#     reg.fit(X_new, y_new)

#     # Set up features on remaining entries in W
#     n = y_train.size
#     X_pair = np.full(shape=(int((n * (n - 1)) / 2), 3 * X_train.shape[1]), fill_value=np.NaN)
#     k = 0

#     # Fill in features for upper triangle of W
#     """ Stopped halfway (upper triangle only) """
#     for i in range(n):
#         for j in range(i + 1, n):
#             X_pair[k] = np.hstack((X_train[i], X_train[j], X_train[i] - X_train[j]))
#             k += 1

#     # Make and scale (since probabilities) predictions on upper triangle of W
#     y_pair = reg.predict(X_pair)
#     y_pair = minmax_scale(y_pair, feature_range=(0, 1))
    
#     # Fill in W symmetrically
#     W = np.full(shape=(n, n), fill_value=np.NaN)
#     k = 0
    
#     for i in range(n):
#         for j in range(i + 1, n):
#             W[i, j] = y_pair[k]
#             W[j, i] = 1 - y_pair[k]
#             k += 1

#     # Fill in diagonal as 0.5
#     W[np.diag_indices(W.shape[0])] = 0.5

#     # Fill in known entries in W symmetrically
#     k = 0
#     for (i, j) in save_idx:
#         W[i, j] = y_new[k]
#         W[j, i] = 1 - y_new[k]
#         k += 1
    
#     # Return W rounded to nearest 0.5 if rounded
#     if rounded:
#         return np.round(W * 2) / 2
#     else:
#         return W

In [633]:
W_new = generate_rank_W(X_train, y_train, m=200, rounded=True)

In [634]:
# # Could also round to 0, 0.5, 1
# W_round = np.round(W_new * 2) / 2
# np.mean((np.abs(W_round - W_true))), np.std((np.abs(W_round - W_true))) 

np.mean(np.abs(W_new - W_true)), np.std(np.abs(W_new - W_true))

(0.16308571428571428, 0.23492288965026945)

In [571]:
s = 0
for i in range(700):
    for j in range(700):
        if W[i, j] != (1 - W[j, i]):
            s += 1

s / (700 ** 2)

0.0

In [451]:
np.sum(np.triu(W_true, k=1)), np.sum(np.tril(W_true, k=1))

122166.5

In [600]:
svm_lambdaboost(X_train, y_train, X_test, y_test, W_round, T=10, sample_prop=4)

LINEAR SVM SUBMODELS
Using 10 classifiers and sample proportion of 4
t	Train		Test
1	0.81		0.81
2	0.29		0.31
3	0.73		0.74
4	0.32		0.34
5	0.71		0.72
6	0.35		0.36
7	0.73		0.72
8	0.38		0.39
9	0.76		0.75
10	0.49		0.50
Done!


(0.4942857142857143, 0.49859681945743684)

In [620]:
tree_lambdaboost(X_train, y_train, X_test, y_test, W_round, T=10, sample_prop=4)

DECISION TREE SUBMODELS
Using 10 classifiers and sample proportion of 4
t	Train		Test
1	0.80		0.80
2	0.21		0.20
3	0.79		0.80
4	0.21		0.20
5	0.79		0.80
6	0.21		0.20
7	0.79		0.80
8	0.21		0.20
9	0.79		0.80
10	0.79		0.80
Done!


(0.20714285714285718, 0.19706891175553476)